In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'elasticsearch', 'port': 9200}])

/usr/local/lib/python2.7/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
import time
def timing(f):
    def wrap(*args):
        time1 = time.time()
        ret = f(*args)
        time2 = time.time()
        print('{:s} function took {:.3f} ms'.format(f.__name__, (time2-time1)*1000.0))

        return ret
    return wrap

In [3]:
es.indices.get_alias("*")

{u'.apm-agent-configuration': {u'aliases': {}},
 u'.kibana_1': {u'aliases': {u'.kibana': {}}},
 u'.kibana_task_manager_1': {u'aliases': {u'.kibana_task_manager': {}}},
 u'montevideo': {u'aliases': {}}}

In [4]:
from elasticsearch.client import IndicesClient
ic = IndicesClient(es)

In [5]:
texts = [
   'Siento mucha tristeza al ver esto en Montevideo. Una persona compartió esta imagen que ocurrió hace minutos en la esquina de Salto y (Bernabé) Rivera. La cantidad de personas en situación de calle creció en la capital. Hay planes para mejorar esta realidad, pero duele verla.',
    'Mucha basura suelta por Varela frente al Policial',
    'En Montevideo podés ir a Porongos esquina Blandengues Upside-down face',
    '18 de Julio esquina Aquiles Lanza, Montevideo, Uruguay',
    'Roban estación de servicio  esquina Montevideo shopping. Hace una semana robaron una joyería a 50 metros de esa estación de servicio. Lugar de mucho tránsito de personas y vehículos. Lleno de camaras de video vigilancia y PADO Man police officerDown pointing backhand index',
    'En esta foto estas en que lugar de Europa, amorosa? Ésta estuvo en el cerro de montevideo, calle Venezuela esquina Bogotá....Jaaaa....sos una genia! Smiling face with open mouth and smiling eyes',
    '👷🏻‍| #MontevideoMejora Round pushpin Av. Italia de Gallinal a Bolivia, luce más amplia y linda Bus Un carril más para el bus Mejor captación de aguas pluviales 🅿 Refugios Bicisenda 40 nuevas columnas de alumbrado y 80 luminarias LED Más info:',
    '@quejasyaEn la esquina de Montevideo Shopping hay una "montaña" de basura (Galarza y Tiburcio Gómez)',
    'Cheering megaphone Modificación de paradas para líneas 147 y 148 hacia afuera Calendar Desde el 5 de marzo Cross mark Se suprime la parada de Paysandú esquina Tristán Narvaja Hourglass Funciona provisoriamente en Tristán Narvaja esquina Cerro Largo',
    'Un hecho histórico en la Medicina Mundial. Ciudad de Montevideo. Esquina de Colonia y Arenal Grande. CASMU 1',
    'Cno. de las Tropas esquina Verdún',
    '¡Hola! Para este tipo de errores en el registro debes acudir a nuestra oficina, allí te darán la información y la ayuda necesaria para corregir este error y completar tu proceso. Recuerda que nuestra oficina está ubicada en Montevideo, José Ellauri 938, esquina Benito Lamas. 🚴🏽Sparkling heart',
    '🚧❌ | MEDIA CALZADA 📍 26 DE MARZO entre L.A. de Herrera y Marco Bruto 🕖 De 7 a 16 h  📍 JOSÉ ELLAURI esquina Leyenda Patria 🕖 De 7 a 17 h  📍 LAGUNILLAS esquina Joaquín Núñez 🕖 De 8 a 18 h  📍 8 DE OCTUBRE desde 18 de Julio hasta Colonia 🕖 De 8 a 18 h',
    'Speaker with three sound waves “Durazno y Convención” es una canción de 1984, que hoy es un tema que se puede escuchar mismo en la esquina sobre la que habla la letraRound pushpin  Ya que es un punto de Montevideo Sonoro Mobile phone Headphone',
    'En la ciudad de Montevideo, hace ya bastante tiempo, están dibujados los hongos de Super Mario Bros. En avenida Uruguay esquina Arenal Grande, está dibujado un hongo rojo. En avenida San Martín esquina Libres, está dibujado un hongo verde. Smiling face with heart-shaped eyes',
    'Alquiler anual. Centro de Montevideo 1 dormitorio $19.000 Héctor Gutiérrez Ruiz esquina San José Coordine visitas al 094283539 Llamadas o wsp',
    '¿Sos marica que usás vestido? Utilizando ésta frase tres efectivos de la Guardia Republicana apalearon a Joel, artista callejero que trabaja habitualmente en Millán y Garzón.'
]

In [6]:
def analyzedText(text):
    results = ic.analyze(index="montevideo", body=
    {
      "text": text, 
        "analyzer":"calle_analyzer"
    })
    return ' '.join([token['token'] for token in results['tokens']])

In [7]:
#@timing
def search(text):
    return es.search(index="montevideo", body=
        {
            "from" : 0, "size" : 500,
              "query": {
                "simple_query_string" : {
                    "query": text,
                    "analyzer": "calle_analyzer"
                }
              }
        })['hits']['hits']

In [8]:
#@timing
def boosting_search(text, size=50):
    return es.search(index="montevideo", body=
{
    "from" : 0, "size" : size,
    "query": {
        "boosting" : {
            "positive" : {
                "simple_query_string" : {
                    "query": text,
                    "analyzer": "calle_analyzer"
                }
            },
            "negative" : {
                "match" : {
                    "type" : "geonames_uy_montevideo limites_barrios"
                }
            },
            "negative_boost" : 0.5
        }
    }
})['hits']['hits']

In [9]:
#@timing
def boosting_match_search(text, size=150):
    return es.search(index="montevideo", body=
{
    "from" : 0, "size" : size,
    "query": {
        "boosting" : {
            "positive" : {
                "multi_match" : {
                    "query": text,
                    "analyzer": "calle_analyzer",
                    "fields": [ "nombre", "aliases" ],
                    "type": "best_fields",
                    
                }
            },
            "negative" : {
                "match" : {
                    "type" : "geonames_uy_montevideo limites_barrios"
                }
            },
            "negative_boost" : 0.5
        }
    }
})['hits']['hits']

In [10]:
#@timing
def search_geo_vias(id,size=150):
    return es.search(index="montevideo", body=
    {
        "from" : 0, "size" : size,
        "query" : {
            "bool": {
                "must": {
                    "match" : {
                        "type" : "v_mdg_vias"
                    }
                },
                "filter": {
                    "geo_shape": {
                        "geometry": {
                            "indexed_shape": {
                                "index": "montevideo",
                                "id": id,
                                "path": "geometry"
                            }
                        }
                    }
                }
            }
        }
    })['hits']['hits']

In [11]:
def stripName(name):
    return name.replace(' ','-').strip().lower()

In [12]:
def getMatchName(result_obj_1,result_obj_2):
    return result_obj_1['s_name'] + '|' + result_obj_2['s_name']

In [13]:
def matchedName(match_dict,result_object):
    matched_key = next((key for key in match_dict.keys() if result_object['s_name'] in key), None)
    return matched_key!=None

In [14]:
def tryToMatchLines(match_dict,line_results,all_results):
    for key, res_obj in line_results.items():
        if matchedName(match_dict, res_obj):
            #If there is alredy a match that involves this name continue
            continue
        for geo_result in search_geo_vias(res_obj['id']):
            #I only care for those objects intersecting current object AND where part of the original results.
            geo_obj = all_results.get(geo_result['_id'], None)
            if geo_obj and res_obj['s_name'] != geo_obj['s_name']:
                #I only need to care for different objects, matching names and not ids.
                #In case of streets more than one block can intersect with the next one, same name diff id.
                match_name = getMatchName(res_obj,geo_obj)
                match_dict[match_name] = [res_obj,geo_obj]
                print ('✔️ MATCH: {}\t/\t{}'.format(match_name,res_obj['score']+geo_obj['score']))

In [15]:
def getResultObject(result):
    result_geo_type = result['_source']['geometry']['type']
    result_id = result['_id']
    result_score = result['_score']
    result_geometry = result['_source']['geometry']
    result_name = 'NO_NAME'
    if result['_source'].get('nombre', None):
            result_name = result['_source']['nombre'].encode('ascii', 'ignore').decode('ascii')
    elif result['_source'].get('aliases',None):
            result_name = result['_source']['aliases'].encode('ascii', 'ignore').decode('ascii')
    result_striped_name = stripName(result_name)
    return {'id':result_id,'geo_type':result_geo_type,'name':result_name, 's_name':result_striped_name,'score':result_score,'geometry':result_geometry}

In [16]:
@timing
def complete_search(text):  
    match_dict = {}
    
    line_results = {}
    point_results = {}
    polygon_results = {}
    all_results = {}

    results = boosting_match_search(text)
    for result in results:
        result_object = getResultObject(result)
        if result_object['geo_type'] == 'LineString':
            line_results[result_object['id']] = result_object
        elif result_object['geo_type'] == 'Point':   
            point_results[result_object['id']] = result_object
        elif result_object['geo_type'] == 'Polygon':
            polygon_results[result_object['id']] = result_object
        else:
            print('Do i have other?')
        
    all_results.update(line_results)
    all_results.update(point_results)
    all_results.update(polygon_results)
    
    tryToMatchLines(match_dict,line_results,all_results)
    
    return results, match_dict

In [17]:
for text in texts:
    print(analyzedText(text))
    results, match_dict = complete_search(text)
    print('\n')

siento mucha tristeza ver montevideo persona compartio imagen ocurrio hace minutos esquina salto bernabe rivera cantidad personas situacion calle crecio capital planes mejorar realidad duele verla
✔️ MATCH: bernabe-michelena|psje-bernabe-michelena	/	15.514187
✔️ MATCH: bernabe-rivera|salto	/	24.545059
complete_search function took 3565.633 ms


mucha basura suelta varela frente policial
complete_search function took 731.422 ms


montevideo podes porongos esquina blandengues upside down face
✔️ MATCH: porongos|blandengues	/	19.765999
complete_search function took 111.426 ms


julio esquina aquiles lanza montevideo uruguay
✔️ MATCH: julio-cesar|4-de-julio	/	10.015156
✔️ MATCH: dr-aquiles-r-lanza|psje-julio-castro	/	17.191444
✔️ MATCH: cno-boiso-lanza|cno-cont-boiso-lanza	/	12.429656
complete_search function took 1165.039 ms


roban estacion servicio esquina montevideo shopping hace semana robaron joyeria estacion servicio lugar transito personas vehiculos lleno camaras video vigilancia p

In [34]:
print(texts[6])
print(analyzedText(texts[6]))
for res in boosting_match_search(texts[6]):
    res = getResultObject(res)
    print(res['score'],res['name'], res['id'])

👷🏻‍| #MontevideoMejora Round pushpin Av. Italia de Gallinal a Bolivia, luce más amplia y linda Bus Un carril más para el bus Mejor captación de aguas pluviales 🅿 Refugios Bicisenda 40 nuevas columnas de alumbrado y 80 luminarias LED Más info:
👷🏻‍ montevideomejora round pushpin italia gallinal bolivia luce mas amplia linda bus carril mas bus mejor captacion aguas pluviales 🅿 refugios bicisenda 40 nuevas columnas alumbrado 80 luminarias led mas info
(19.458035, u'GREGORIO MAS DE AYALA', u'v_mdg_vias-5136802')
(19.458035, u'GREGORIO MAS DE AYALA', u'v_mdg_vias-6619184')
(19.458035, u'GREGORIO MAS DE AYALA', u'v_mdg_vias-6834690')
(19.458035, u'GREGORIO MAS DE AYALA', u'v_mdg_vias-6834691')
(19.458035, u'JOSE MAS DE AYALA', u'v_mdg_vias-6877344')
(17.228832, u'DR ISIDRO MAS DE AYALA', u'v_mdg_vias-7090394')
(17.228832, u'DR ISIDRO MAS DE AYALA', u'v_mdg_vias-7090418')
(17.228832, u'DR ISIDRO MAS DE AYALA', u'v_mdg_vias-7090420')
(17.228832, u'DR ISIDRO MAS DE AYALA', u'v_mdg_vias-7090421')

In [19]:
analyzedText('avenida italia calle 1 av espana')

u'italia espana'

In [412]:
for res in boosting_search('uruguay',80):
    res = getResultObject(res)
    print(res['score'],res['name'], res['id'])

(13.179165, u'AV URUGUAY', u'v_mdg_vias-287567')
(13.179165, u'AV URUGUAY', u'v_mdg_vias-6620887')
(13.179165, u'AV URUGUAY', u'v_mdg_vias-6621692')
(13.179165, u'AV URUGUAY', u'v_mdg_vias-6621693')
(13.179165, u'AV URUGUAY', u'v_mdg_vias-7120202')
(13.179165, u'AV URUGUAY', u'v_mdg_vias-7120203')
(13.179165, u'AV URUGUAY', u'v_mdg_vias-293015')
(13.179165, u'AV URUGUAY', u'v_mdg_vias-294443')
(13.179165, u'AV URUGUAY', u'v_mdg_vias-295744')
(13.179165, u'AV URUGUAY', u'v_mdg_vias-296476')
(13.179165, u'AV URUGUAY', u'v_mdg_vias-297300')
(13.179165, u'AV URUGUAY', u'v_mdg_vias-298104')
(13.179165, u'AV URUGUAY', u'v_mdg_vias-298830')
(13.179165, u'AV URUGUAY', u'v_mdg_vias-299764')
(13.179165, u'AV URUGUAY', u'v_mdg_vias-300414')
(13.179165, u'AV URUGUAY', u'v_mdg_vias-292483')
(13.179165, u'AV URUGUAY', u'v_mdg_vias-292585')
(13.179165, u'AV URUGUAY', u'v_mdg_vias-288716')
(13.179165, u'AV URUGUAY', u'v_mdg_vias-289020')
(13.179165, u'AV URUGUAY', u'v_mdg_vias-289027')
(13.179165, u'A

In [360]:
for res in search(text):
    res = getResultObject(res)
    print(res['score'],res['name'], res['id'])

(30.083754, u'Colonia Nicolich', u'geonames_uy_montevideo-3480820')
(30.002186, u'Facultad de Medicina - UdelaR', u'geonames_uy_montevideo-7290179')
(27.47543, u'ARENAL GRANDE', u'v_mdg_vias-7365550')
(27.47543, u'ARENAL GRANDE', u'v_mdg_vias-7365553')
(27.47543, u'ARENAL GRANDE', u'v_mdg_vias-7365555')
(27.47543, u'ARENAL GRANDE', u'v_mdg_vias-7365557')
(27.47543, u'ARENAL GRANDE', u'v_mdg_vias-7365559')
(27.47543, u'ARENAL GRANDE', u'v_mdg_vias-7365560')
(27.47543, u'ARENAL GRANDE', u'v_mdg_vias-7365561')
(27.47543, u'ARENAL GRANDE', u'v_mdg_vias-7369992')
(27.47543, u'ARENAL GRANDE', u'v_mdg_vias-7369993')
(27.47543, u'ARENAL GRANDE', u'v_mdg_vias-7370006')
(27.47543, u'ARENAL GRANDE', u'v_mdg_vias-7370030')
(27.47543, u'ARENAL GRANDE', u'v_mdg_vias-287131')
(27.47543, u'ARENAL GRANDE', u'v_mdg_vias-287362')
(27.47543, u'ARENAL GRANDE', u'v_mdg_vias-287367')
(27.47543, u'ARENAL GRANDE', u'v_mdg_vias-287596')
(27.47543, u'ARENAL GRANDE', u'v_mdg_vias-287621')
(27.47543, u'ARENAL GRAN